<a href="https://colab.research.google.com/github/dtphuong2612/defect_detection/blob/main/training_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install segmentation_models

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 1.5 MB/s eta 0:00:00


In [ ]:
from sklearn.model_selection import train_test_split
import os
import random

import tensorflow as tf
import cv2
import numpy as np

# Import thu vien segmentation_models
from segmentation_models.metrics import iou_score
from segmentation_models import Unet
import segmentation_models as sm
sm.set_framework("tf.keras")
sm.framework()

# Dinh nghia bien
data_path  = "/kaggle/input/dataset-defect/NV_public_defects"
w, h = 512, 512
batch_size = 16

# Dataset va Dataloader

BACKBONE = "inceptionv3"
preprocess_input = sm.get_preprocessing(BACKBONE)

In [ ]:


# Dung de tao toan bo du lieu va load theo batch
class Dataset:
    def __init__(self, image_path, mask_path, w, h):
        # the paths of images
        self.image_path = image_path
        # the paths of segmentation images
        self.mask_path = mask_path

        self.w = w
        self.h = h

    def __getitem__(self, i):
        # read data
        image = cv2.imread(self.image_path[i])
        image = cv2.resize(image, (self.w, self.h), interpolation=cv2.INTER_AREA)
        image = preprocess_input(image)

        mask = cv2.imread(self.mask_path[i], cv2.IMREAD_UNCHANGED)
        image_mask = cv2.resize(mask, (self.w, self.h), interpolation=cv2.INTER_AREA)

        image_mask = [(image_mask == v) for v in [1]]
        image_mask = np.stack(image_mask, axis=-1).astype('float')

        return image, image_mask

class Dataloader(tf.keras.utils.Sequence):
    def __init__(self, dataset, batch_size,shape, shuffle=False):
        self.dataset = dataset
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.shape = shape
        self.indexes = np.arange(self.shape)

    def __getitem__(self, i):
        # collect batch data
        start = i * self.batch_size
        stop = (i + 1) * self.batch_size
        data = []
        for j in range(start, stop):
            data.append(self.dataset[j])

        batch = [np.stack(samples, axis=0) for samples in zip(*data)]

        return tuple(batch)

    def __len__(self):
        return len(self.indexes) // self.batch_size

    def on_epoch_end(self):
        if self.shuffle:
            self.indexes = np.random.permutation(self.indexes)

#  Load thong tin tu folder dataset de tao 2 bien image_path, mask_path
def load_path(data_path):
    # Get normal image and mask
    classes = ['Class1', 'Class2', 'Class3', 'Class4', 'Class5', 'Class6']

    # Lop qua cac thu muc khong loi
    normal_image_path = []
    normal_mask_path = []
    for class_ in classes:
        current_folder = os.path.join(data_path, class_)
        for file in os.listdir(current_folder):
            if file.endswith("png") and (not file.startswith(".")):
                image_path = os.path.join(current_folder, file)
                mask_path = os.path.join(current_folder + "_mask", file)
                normal_mask_path.append(mask_path)
                normal_image_path.append(image_path)

    # Get defect image and mask
    defect_image_path = []
    defect_mask_path = []
    for class_ in classes:
        class_ = class_ + "_def"
        current_folder = os.path.join(data_path, class_)
        for file in os.listdir(current_folder):
            if file.endswith("png") and (not file.startswith(".")):
                image_path = os.path.join(current_folder, file)
                mask_path = os.path.join(current_folder + "_mask", file)
                defect_mask_path.append(mask_path)
                defect_image_path.append(image_path)

    # Normal:   normal_mask_path - chua toan bo duong dan cua mask
    #            normal_image_path-  chua toan bo duong dan den image
    # Defect:   defect_mask_path - chua toan bo duong dan cua mask
    #            defect_image_path-  chua toan bo duong dan den image

    idx = random.sample(range(len(normal_mask_path)), len(defect_mask_path))

    normal_mask_path_new = []
    normal_image_path_new = []

    for id in idx:
        normal_image_path_new.append(normal_image_path[id])
        normal_mask_path_new.append(normal_mask_path[id])

    image_path = normal_image_path_new + defect_image_path
    mask_path = normal_mask_path_new + defect_mask_path

    return image_path, mask_path

# Thu hien load va train model

# Load duong dan vao 2 bien
image_path, mask_path = load_path(data_path)

# Chia du lieu train, test
image_train, image_test, mask_train, mask_test = train_test_split(image_path, mask_path, test_size=0.2)

# Tao dataset va dataloader
train_dataset = Dataset(image_train, mask_train, w, h)
test_dataset = Dataset(image_test, mask_test, w, h)

train_loader = Dataloader(train_dataset, batch_size, shape=len(image_train), shuffle=True)
test_loader = Dataloader(test_dataset, batch_size, shape=len(image_test), shuffle=True)


In [ ]:
wgt = '/kaggle/input/weight-inceptionv3/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'
# Khoi tao model
opt=tf.keras.optimizers.Adam(0.01)
model= Unet(BACKBONE,encoder_weights=wgt,classes=1,activation="sigmoid",input_shape=(512,512,3),encoder_freeze=True)
loss1 = sm.losses.categorical_focal_dice_loss
model.compile(optimizer=opt,loss=loss1,metrics=[iou_score])

# Train model
from keras.callbacks import ModelCheckpoint
filepath="ckp_resnet50.hdf5"
callback = ModelCheckpoint(filepath, monitor='val_iou_score', verbose=1, save_best_only=True,mode='max')

model.fit_generator( train_loader, validation_data=test_loader, epochs=60, callbacks=[callback])


Epoch 1/60
90/90 [==============================] - 63s 601ms/step - loss: 0.6401 - iou_score: 0.2498 - val_loss: 0.9726 - val_iou_score: 0.0147

Epoch 00001: val_iou_score improved from -inf to 0.01471, saving model to ckp_resnet50.hdf5
Epoch 2/60
90/90 [==============================] - 52s 571ms/step - loss: 0.3552 - iou_score: 0.4957 - val_loss: 0.7987 - val_iou_score: 0.1156

Epoch 00002: val_iou_score improved from 0.01471 to 0.11560, saving model to ckp_resnet50.hdf5
Epoch 3/60
90/90 [==============================] - 52s 579ms/step - loss: 0.2933 - iou_score: 0.5646 - val_loss: 0.4006 - val_iou_score: 0.4463

Epoch 00003: val_iou_score improved from 0.11560 to 0.44628, saving model to ckp_resnet50.hdf5
Epoch 4/60
90/90 [==============================] - 52s 575ms/step - loss: 0.2579 - iou_score: 0.6066 - val_loss: 0.3448 - val_iou_score: 0.5019

Epoch 00004: val_iou_score improved from 0.44628 to 0.50187, saving model to ckp_resnet50.hdf5
Epoch 5/60
90/90 [=====================